In [1]:
import os
import glob
import numpy as np
import pandas as pd
import openslide
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt 

from utils import Reinhard, get_TC
from utils_vis import get_GAPCAM, get_gradCAM, plot_oneline

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

2025-05-06 05:05:41.526125: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 05:05:41.811744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746504342.018577 2120203 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746504342.116381 2120203 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746504342.315762 2120203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
patch_size = 1024
model_weights = './data/TC_1024px.h5'
model = get_TC(model_weights, image_size=(patch_size, patch_size), num_classes=3, output_features=False)

/cephfs/volumes/hpc_data_prj/cb_histology_data/8b8186f5-4ae0-4605-94c4-46d4ff4e62b0/Siyuan/Docker_test/nbtclassifier/NBT-Classifier/utils.py:41: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  net = MobileNet(include_top=False, input_shape=(image_size[0], image_size[1], 3))
I0000 00:00:1746504365.233935 2120203 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1230 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:17:00.0, compute capability: 8.0


## CAMs

In [3]:
img_path = "../examples/patch_examples/T18-08826 I1 HE [x=19456,y=90112,w=1024,h=1024].png"
channel = 0
pred_prob, img, CAM, cam_heatmap, super_imposed_cam = get_GAPCAM(img_path = img_path, model = model, layer='conv_pw_13_relu', 
                                                            label_index=channel, with_grids=False, patch_size=patch_size)

plot_oneline(img_list=[img, CAM, cam_heatmap, super_imposed_cam], 
             caption_list=[f'channel_{channel}','',pred_prob,''],
             figure_size=(10, 5),  save_pt=None) 

/scratch/users/k21066795/conda/nbtclassifier/lib/python3.9/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor']]
Received: inputs=Tensor(shape=(1, 1024, 1024, 3))
  warnings.warn(msg)
I0000 00:00:1746504389.248153 2120203 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-06 05:06:49.976241: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00MiB (rounded to 8388608)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-06 05:06:49.976278: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-06 05:06:49.976296: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Tota

ResourceExhaustedError: Exception encountered when calling BatchNormalization.call().

[1m{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: [0m

Arguments received by BatchNormalization.call():
  • inputs=tf.Tensor(shape=(1, 64, 64, 512), dtype=float32)
  • training=None
  • mask=None

In [ ]:
img_path = "../examples/patch_examples/T18-08826 I1 HE [x=28672,y=67584,w=1024,h=1024].png"
channel = 2
pred_prob, img, CAM, cam_heatmap, super_imposed_cam = get_GAPCAM(img_path = img_path, model = model, layer='conv_pw_13_relu', 
                                                            label_index=channel, with_grids=False, patch_size=patch_size)

plot_oneline(img_list=[img, CAM, cam_heatmap, super_imposed_cam], 
             caption_list=[f'channel_{channel}','',pred_prob,''],
             figure_size=(10, 5),  save_pt=None) 

In [ ]:
img_path = "../examples/patch_examples/T18-08826 I1 HE [x=69632,y=38912,w=1024,h=1024].png"
channel = 1
pred_prob, img, CAM, cam_heatmap, super_imposed_cam = get_GAPCAM(img_path = img_path, model = model, layer='conv_pw_13_relu', 
                                                            label_index=channel, with_grids=False, patch_size=patch_size)

plot_oneline(img_list=[img, CAM, cam_heatmap, super_imposed_cam], 
             caption_list=[f'channel_{channel}','',pred_prob,''],
             figure_size=(10, 5),  save_pt=None) # f"{save_pt}{channel}"

## Grad-CAMs

In [ ]:
img_path = "../examples/patch_examples/T18-08826 I1 HE [x=19456,y=90112,w=1024,h=1024].png"
channel = 0
pred_prob, img, CAM, cam_heatmap, super_imposed_cam = get_gradCAM(img_path = img_path, model = model, layer='conv_pw_13_relu', 
                                                            label_index=channel, with_grids=False, patch_size=patch_size)

plot_oneline(img_list=[img, CAM, cam_heatmap, super_imposed_cam], 
             caption_list=[f'channel_{channel}','',pred_prob,''],
             figure_size=(10, 5),  save_pt=None) # f"{save_pt}{channel}"

In [ ]:
img_path = "../examples/patch_examples/T18-08826 I1 HE [x=28672,y=67584,w=1024,h=1024].png"
channel = 2
pred_prob, img, CAM, cam_heatmap, super_imposed_cam = get_GAPCAM(img_path = img_path, model = model, layer='conv_pw_13_relu', 
                                                            label_index=channel, with_grids=False, patch_size=patch_size)

plot_oneline(img_list=[img, CAM, cam_heatmap, super_imposed_cam], 
             caption_list=[f'channel_{channel}','',pred_prob,''],
             figure_size=(10, 5),  save_pt=None) # f"{save_pt}{channel}"

In [ ]:
img_path = "../examples/patch_examples/T18-08826 I1 HE [x=69632,y=38912,w=1024,h=1024].png"
channel = 1
pred_prob, img, CAM, cam_heatmap, super_imposed_cam = get_gradCAM(img_path = img_path, model = model, layer='conv_pw_13_relu', 
                                                            label_index=channel, with_grids=False, patch_size=patch_size)

plot_oneline(img_list=[img, CAM, cam_heatmap, super_imposed_cam], 
             caption_list=[f'channel_{channel}','',pred_prob,''],
             figure_size=(10, 5),  save_pt=None) # f"{save_pt}{channel}"